# ETL pipeline and Data warehouse using Python and Postgresql

Here, I will create a simple ETL pipeline using Python and Postgresql and won't use AWS. 

### Basic ETL Workflow
1. Extract data from CSV files 
2. Transform the data using Pandas
3. Load the transformed data to a data warehouse created using Postgresql

### For Data warehouse
- Create Data Model illustrating star schema with fact and dimension tables

In [1]:
#* Imports 
# sqlalchemy and psycopg2 will be used to connect to the postgresql database
from sqlalchemy import create_engine
import psycopg2
# pygrametl is used to create ETL flow
import pygrametl 
# To read data from external csv files
from pygrametl.datasources import CSVSource
# Using pygrametl we can interact with dimensions and fat table using set of classes
from pygrametl.tables import CachedDimension, FactTable 
# For data processing
import pandas as pd 
# For data visualization
import matplotlib.pyplot as plt